In [1]:
import sys
import numpy as np
np.set_printoptions(suppress=True, precision=4)
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.4f}'.format
import seaborn as sns
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import IsolationForest
from sklearn.metrics import log_loss, make_scorer
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.metrics import BinaryCrossentropy
from methods import *
import torch
import pickle, os
import skopt
from skopt import BayesSearchCV, gp_minimize
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
import shutil
import warnings
warnings.filterwarnings("ignore")
print('done')

done


In [2]:
%load_ext autoreload
%autoreload 2
print('done')

done


# Explore dataset

In [ ]:
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-50-random_filters.csv'
path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-30-gray_filters_h_kl.csv'
df = pd.read_csv(path_csv)
df.describe()

In [ ]:
X, y = read_features(path_csv, trigger_type_aux_str=None, arch=None, data='hkl', append_arch=False)
# print(X[0,:])
i = 12
f1, f2 = X[:, i], X[:, i+1]
plt.figure(figsize=(20,15)).patch.set_color('white')
plt.scatter(f1, f2, c=[['blue', 'red'][vy] for vy in y], s=20.0)
plt.grid()

## Correlation Plot

In [ ]:
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-filters_all-classes_gray.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
# path_csv = fr'confusion-reports\ics_svm\round2-holdout-dataset\round2-holdout-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-30-gray_filters_h_kl.csv'
df = pd.read_csv(path_csv)
df['model_label'] = 1 - df['model_label'].astype('category').cat.codes
del df['num_classes']
indexes = []
arch = None
if arch is not None:
    for i in range(len(df)):
        if df['model_architecture'].iloc[i].startswith(arch):
            indexes.append(i)
    df = df.iloc[indexes]
for c in df.columns:
    if c.endswith('mean') or c.endswith('std') or c.startswith('h'):
#     if c.endswith('diff'):
        del df[c]
print(df.shape)
print(df.iloc[0])
plt.figure(figsize=(30,12)).patch.set_color('white')
sns.heatmap(df.corr(), annot=True, cmap='YlGnBu')

# plt.figure(figsize=(30,12)).patch.set_color('white')
# sns.pairplot(df, hue='model_label')

## Box-plots

In [ ]:
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-filters_all-classes_gray.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
# path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square30-gray_filters_h_kl.csv'
df = pd.read_csv(path_csv)
df['model_label'] = 1 - df['model_label'].astype('category').cat.codes
del df['num_classes'], df['model_name']
indexes = []
arch = None
if arch is not None:
    for i in range(len(df)):
        if df['model_architecture'].iloc[i].startswith(arch):
            indexes.append(i)
    df = df.iloc[indexes]
for c in df.columns:
    if c.endswith('mean') or c.endswith('std'):
#     if c.endswith('diff'):
        del df[c]
del df['model_architecture'], df['architecture_code'], df['trigger_type_aux'], df['trigger_color']#, df['model_label']
print(df.columns.tolist())
for c in df.columns:
#     if c.startswith('h'):
    if c.endswith('mean_diff'):
        plt.figure().patch.set_color('white')
        sns.boxplot(x="model_label", y=c, hue="model_label", data=df, palette="Set3")
print('done')

In [ ]:
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-filters_all-classes_gray.csv'
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
df = pd.read_csv(path_csv)
df['model_label'] = 1 - df['model_label'].astype('category').cat.codes
indexes = []
arch = None
if arch is not None:
    for i in range(len(df)):
        if df['model_architecture'].iloc[i].startswith(arch):
            indexes.append(i)
    df = df.iloc[indexes]
for c in df.columns:
    if c.endswith('mean') or c.endswith('std'):
#     if c.endswith('diff'):
        del df[c]
del df['model_architecture'], df['trigger_type_aux'], df['trigger_color'], df['num_classes'], df['model_name']
# print(df.head())

data = df.values
X, y = data[:, 1:], data[:, 0]
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X)
mask = np.where(yhat == 1)[0]
print(mask.shape)
del df['model_label']
plt.figure(figsize=(30, 12)).patch.set_color('white')
df.iloc[mask].boxplot()

# Stratified K-fold validation for a full training dataset (square-size and 5 filters)

In [10]:
# path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square25-filters_triggered_classes.csv' # old dataset
# path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray.csv'
# path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
n_splits = 10
n_repeats = 3

params = {
    "C": [0.01 + i * 0.01 for i in range(1000)]
}

for size in [30]: #[10, 15, 20, 25, 30, 35, 40, 45, 50]:
#     path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}_filters.csv'
#     path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}-random_filters.csv'
    path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-{size}-gray_filters.csv'
#     path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-{size}-random_filters.csv'
#     path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-{size}-gray_filters_h_kl.csv'
#     path_csv = fr'confusion-reports/conf_mat/round3-train-dataset_fc_square-{size}-random_filters_h_kl.csv'
    for A in [None]: #['vgg', 'resnet', 'wideresnet', 'densenet', 'mobilenet', 'shufflenet', 'squeezenet', 'inception', 'googlenet', None]:
        X, y = read_features(path_csv, trigger_type_aux_str=None, arch=A, data='diffs', append_arch=False, arch_one_hot=False)
        X, y = X[2:,:], y[2:]
        scores_roc, scores_xent = [], []
        print(X.shape, y.shape)
        print(X[0,:].tolist())
        try:
            for _ in range(n_repeats):
                kfold =  StratifiedKFold(n_splits=n_splits, shuffle=True)
                for train_index, test_index in kfold.split(X, y):
                    X_train, y_train = X[train_index, :], y[train_index]
                    X_test, y_test = X[test_index, :], y[test_index]
    #                 scaler = StandardScaler()
    #                 scaler.fit(X_train)
    #                 X_train = scaler.transform(np.copy(X_train))
    #                 X_test = scaler.transform(np.copy(X_test))
                    roc, xent = evaluate_classifier(RandomForestClassifier(n_estimators=500), X_train, y_train, X_test, y_test)
                    scores_roc.append(roc)
                    scores_xent.append(xent)
        except ValueError:
            print('ERROR')
        print(f'size={size}')
        print(f'CE mean={np.mean(scores_xent):.3f}, CE std={np.std(scores_xent):.3f}')
        print('-------------------------------------------------------------------------')
print('done')

Using diffs
(1006, 12) (1006,)
[0.2813518047332764, 0.10623383522033693, 2.644113063812256, 0.007863640785217285, 2.328124046325684, 0.0877617597579956, 1.308053970336914, 0.16514909267425534, 1.3560094833374023, 0.3523577451705933, 2.3160691261291504, 0.2683547735214233]
size=30
CE mean=0.509, CE std=0.044
-------------------------------------------------------------------------
done


# HPO for training data

In [ ]:
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30_filters.csv'
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-random_filters.csv'
X, y = read_features(path_csv, trigger_type_aux_str=None, arch=None, data='diffs', append_arch=True)
print(X.shape)
params = {
    "C": [0.01 + i * 0.01 for i in range(1000)]
}
grid_cv = GridSearchCV(estimator=LogisticRegression(),
                       param_grid=params,
                       cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1),
                       scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True),
                       n_jobs=-1)
grid_cv.fit(X, y)

# print('Best params:', grid_cv.best_params_)
# print('Best CE:', grid_cv.best_score_)
pd.DataFrame.from_dict({
    'C': [d['C'] for d in grid_cv.cv_results_['params']],
    'CE': -grid_cv.cv_results_['mean_test_score']
}).sort_values(['CE'], ascending=True, axis=0)

# Train meta-model using square data and filters

In [ ]:
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'
# path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv'
# path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-random_filters.csv'

X_train, y_train = read_features(path_csv, trigger_type_aux_str=None, arch=None, data='diffs', append_arch=False, arch_one_hot=False)
X_train = X_train[2:, :]
y_train = y_train[2:]
print(X_train.shape)
scaler=None
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)

model = RandomForestClassifier(n_estimators=500)
model.fit(X_train, y_train)

name = '../metamodels/metamodel_13_fc_round3_data=diffs_square=30-gray_scaler=no_clf=rf-500_arch-features=no_exclude-sts=yes'
if os.path.isdir(name):
    shutil.rmtree(name)
    os.mkdir(name)
save_obj(model, f'{name}/model.pickle')
save_obj(scaler, f'{name}/scaler.pickle')


print('done')

In [ ]:
path_csv = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-gray_filters.csv'

X, y = read_features(path_csv, trigger_type_aux_str=None, arch=None, data='diffs')
X_train, y_train = X[:500, :], y[:500]
X_test, y_test = X[500:, :], y[500:]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
model = LogisticRegression(C=1)
model.fit(X_train, y_train)
ce = 0.0
for i in range(X_test.shape[0]):
    prob = model.predict_proba(X_test[i,:].reshape(1, -1))
    p0, p1 = prob[0]
    t = y[i]
    local_ce = -(t * np.log2(p1) + (1-t) * np.log2(1-p0))
#     print(i, local_ce)
    ce += local_ce
print(ce / X_test.shape[0])

# Train Neural Network

In [ ]:
def create_neural_network():
    model = Sequential()
    model.add(Dense(60, input_dim=14, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def cross_validation(data_x, data_y):
#     pipeline = Pipeline([('scale', MinMaxScaler()), ('mlp', KerasClassifier(build_fn=create_neural_network, epochs=100, batch_size=32, verbose=0))])
#     for _ in range(3):
#         pipeline = Pipeline([('mlp', KerasClassifier(build_fn=create_neural_network, epochs=50, batch_size=16, verbose=0))])
#         outer_kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=666)
#         for idx_train_val, idx_test in outer_kfold.split(data_x, data_y):
#             subset_data_x = data_x[idx_train_val, :]
#             subset_data_y = data_y[idx_train_val]
#             inner_kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=666)
    pipeline = Pipeline([('mlp', KerasClassifier(build_fn=create_neural_network, epochs=400, batch_size=32, verbose=0))])
    kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=666)
    results = cross_val_score(pipeline, data_x, data_y, cv=kfold, scoring='neg_log_loss', n_jobs=-1)
    results = results[~np.isnan(results)]
    results = results[~np.isinf(results)]
    print(f'CV: CE mean={-results.mean():.3f} CE std={results.std():.3f}')
    
def train_nn(data_x, data_y, test_x=None, test_y=None):
    model = create_neural_network()
    scaler = MinMaxScaler()
    scaler.fit(data_x)
    data_x_scaled = scaler.transform(data_x)
    history = model.fit(data_x_scaled, data_y, epochs=100, batch_size=32, verbose=0)
    
    loss, accuracy = model.evaluate(data_x_scaled, data_y, verbose=0)
    print(f'Train: loss={loss:.3f} accuracy={accuracy:.3f}')
    
    if test_x is not None and test_y is not None:
        loss, accuracy = model.evaluate(test_x, test_y, verbose=0)
        print(f'Test: loss={loss:.3f} accuracy={accuracy:.3f}')
        
#     folder = r'D:\Cloud\MEGA\TrojAI\TrojAI-UMD\metamodels\metamodel_10_round3_NN-60-30_min-max-scaled_RAW_square30_RANDOM_filters_all-classes'
    folder = r'D:\Cloud\MEGA\TrojAI\TrojAI-UMD\metamodels\metamodel_11_round3_NN-60-30_min-max-scaled_RAW_square30_GRAY'
    keras_save(model, folder)
    save_obj(scaler, os.path.join(folder, 'scaler.pickle'))

path_csv = r'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-filters_all-classes_gray.csv'
X, y = read_features_raw(path_csv, None)

# n = 800
# X_train, y_train = X[:n, :], y[:n]
# X_test, y_test = X[n:, :], y[n:]
# print(X_train.shape, y_train.shape) #, X_test.shape, y_test.shape)


# cross_validation(X, y)
train_nn(X, y)
print('done')

# Local Testing: train on training data and test on holdout data

In [ ]:
path_train_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray.csv'
path_holdout_csv = r'confusion-reports\ics_svm\round2-holdout-dataset\round2-holdout-dataset_square-25-filters_all-classes_gray.csv'
# path_train_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
# path_holdout_csv = r'confusion-reports\ics_svm\round2-holdout-dataset\round2-holdout-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
trigger_type_aux_str = None

print('Local Testing')
if 'confusion-matrix' in path_train_csv and 'confusion-matrix' in path_holdout_csv:
    print('Approach: confusion matrix and original CNN')
    X_train, y_train = read_features_confusion_matrix(path_train_csv, trigger_type_aux_str)
    X_holdout, y_holdout = read_features_confusion_matrix(path_holdout_csv, trigger_type_aux_str)
else:
    print('Approach: confusion distribution and SDNs')
    X_train, y_train = read_features(path_train_csv, trigger_type_aux_str)
    X_holdout, y_holdout = read_features(path_holdout_csv, trigger_type_aux_str)

print('train shape:', X_train.shape, y_train.shape)
print('holdout shape:', X_holdout.shape, y_holdout.shape)

roc, xent = evaluate_classifier(X_train, y_train, X_holdout, y_holdout)
print(f'ROC AUC = {roc}')
print(f'Cross-Entropy = {xent}')